## Face Recognition based on facial features

This program consists of three main phases


1. Detect the individual face in the image and extract facial features using pre-trained models
2. Store people's information in the dataset, including the name, image and feature extracted from their face in the image
3. Compare the extracted features

> 1. Compare the similarity of each person's face with another person in the dataset
> 2. Compare the faces in the new image with the faces of the people in the dataset and get their similarities



# 1. **Face Detection** :
    
   Model : Multi-Task Cascaded Convolutional Neural Network (MTCNN)

In [ ]:
!pip install Pillow

In [ ]:
!pip install mtcnn

In [4]:
from os import listdir
from os.path import isdir
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN

# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
	# load image from file
	image = Image.open(filename)
	# convert to RGB, if needed
	image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

# load images and extract faces for all images in a directory
def load_faces(directory):
	faces = list()
	# enumerate files
	for filename in listdir(directory):
		# path
		path = directory + filename
		# get face
		face = extract_face(path)
		# store
		faces.append(face)
	return faces

# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
	X, y = list(), list()
	# enumerate folders, on per class
	for subdir in listdir(directory):
		# path
		path = directory + subdir + '/'
		# skip any files that might be in the dir
		if not isdir(path):
			continue
		# load all faces in the subdirectory
		faces = load_faces(path)
		# create labels
		labels = [subdir for _ in range(len(faces))]
		# summarize progress
		print('>loaded %d examples for class: %s' % (len(faces), subdir))
		# store
		X.extend(faces)
		y.extend(labels)
	return asarray(X), asarray(y)

In [ ]:
from numpy import savez_compressed

# load dataset
X, y = load_dataset('/content/drive/MyDrive/Git_Repos/FaceRecognition_FaceNet_repo/faces-dataset/data/')
print(X.shape, y.shape)
# save arrays to one file in compressed format
savez_compressed('/content/drive/MyDrive/Git_Repos/FaceRecognition_FaceNet_repo/dataset.npz', X, y)

In [ ]:
X, y = load_dataset('/content/drive/MyDrive/Git_Repos/FaceRecognition_FaceNet_repo/faces-dataset/data/')
print(X.shape, y.shape)

# 2. Extract face embedding using FaceNet
FaceNet is a face recognition system developed in 2015 by researchers at Google that achieved then state-of-the-art results on a range of face recognition benchmark datasets.

The FaceNet model can be used to extract high-quality features from faces, called face embeddings. 

Face embedding actually is a vector with length 128 consist of numeric data.

In [ ]:
from keras.models import load_model

# load the facenet model
model = load_model('/content/drive/MyDrive/Git_Repos/FaceRecognition_FaceNet_repo/facenet_keras.h5')
print('Loaded Model')

In [8]:
from numpy import expand_dims

# calculate a face embedding for each face in the dataset using facenet
# get the face embedding for one face
def get_embedding(model, face_pixels):
	# scale pixel values
	face_pixels = face_pixels.astype('float32')
	# standardize pixel values across channels (global)
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	# transform face into one sample
	samples = expand_dims(face_pixels, axis=0)
	# make prediction to get embedding
	yhat = model.predict(samples)
	return yhat[0]

Note:

If you see lots of warning after running the following cell.
These warnings are related to the implementation of tensorflow in the Google colab and there is no specific problem.

The warnings will not hurt performance.

In [ ]:
from numpy import load
from numpy import asarray
from numpy import savez_compressed

# load the face dataset
data = load('/content/drive/MyDrive/Git_Repos/FaceRecognition_FaceNet_repo/dataset.npz')
X, y = data['arr_0'], data['arr_1']
print('Loaded: ', X.shape, y.shape)
# convert each face in the  dataset to an embedding
newX = list()
for face_pixels in X:
	embedding = get_embedding(model, face_pixels)
	newX.append(embedding)
newX = asarray(newX)
print(newX.shape)
# save arrays to one file in compressed format
savez_compressed('/content/drive/MyDrive/Git_Repos/FaceRecognition_FaceNet_repo/embeddings.npz', newX, y)

# 3. Comparing facial features with using cosine_similarity metric

In [ ]:
#@title compare face features with new image and recoginize person with our dataset(.npz) (without using database) { form-width: "1px" }
from scipy.spatial import distance
import numpy as np

#from sklearn.metrics.pairwise import cosine_similarity
newImage = "/content/drive/MyDrive/Git_Repos/FaceRecognition_FaceNet_repo/faces-dataset/test/leonardo_dicaprio_640.jpg"
face = extract_face(newImage)
embedding = get_embedding(model, face)

# load the face dataset
data = load('/content/drive/MyDrive/Git_Repos/FaceRecognition_FaceNet_repo/embeddings.npz')
X, y = data['arr_0'], data['arr_1']
#print('Loaded: ', X.shape, y.shape)

# compare face with each faces in data set
result = list()
for embedings in X:
    cosine_similarity = 1 - distance.cosine(embedings, embedding)
	#cosine_similarity = cosine_similarity(embedings, embedding, dense_output=True)
    result.append(cosine_similarity)
print(type(result))
print(result)

#out put of recognizing
if max(result) > 0.88 :
    num_person = result.index(max(result))
    print(y[num_person])
else:
    print("this person is not recoginized")


# **End**